## Using GPT model and direct prompt

In [ ]:



import os
import re
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from IPython.display import display

# --- Load Environment Variables ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# --- Initialize LLM ---
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=api_key
)

# --- Prompt Template ---
template = """
You are a qualified financial auditor.

Your task is to **mathematically verify** the totals and subtotals in the following financial table: **{table_name}**.

### Your rules:
- ✅ ONLY check **calculation logic** — no formatting issues like commas.
- ✅ DO NOT mark a row as incorrect if the **Expected and Found values are exactly the same** (even if repeated).
- ❌ DO NOT include any row where values are equal.
- ✅ ONLY report rows where the **Expected ≠ Found**.
- ⚠️ If all calculations are correct, just say so.

---

### Output Format:

- If table is correct:
    Table Name: {table_name}, Status: correct

- If table has errors:
    Table Name: {table_name}, Status: wrong
    Table Name: {table_name}, Row Index: <row index>, Expected: <expected value>, Found: <found value>

📌 Notes:
- Use 0-based indexing.
- No explanations.
- No correct rows.
- No hallucinations — compare numerically.

---

### Table for Validation:
{table_markdown}

Begin validation:
"""

prompt_template = ChatPromptTemplate.from_template(template)

# --- Validation Core ---
def validate_table(table_name: str, df: pd.DataFrame):
    table_markdown = df.to_markdown(index=True, tablefmt="grid")
    prompt = prompt_template.format_messages(table_name=table_name, table_markdown=table_markdown)
    
    response = llm(prompt)
    result = response.content.strip()
    print(f"\n🔍 LLM Response for {table_name}:\n{result}")

    status_match = re.search(r"Status\s*:\s*(correct|wrong)", result, re.IGNORECASE)
    status = status_match.group(1).capitalize() if status_match else "Unknown"

    errors = []
    if status.lower() == "wrong":
        row_pattern = r"Row Index\s*:\s*(\d+),\s*Expected\s*:\s*([-\d,.]+)[,|\s]+(?:Found|Expected)\s*:\s*([-\d,.]+)"
        for match in re.finditer(row_pattern, result, re.IGNORECASE):
            try:
                idx = int(match.group(1))
                exp_raw, found_raw = match.group(2), match.group(3)
                exp = float(exp_raw.replace(",", ""))
                found = float(found_raw.replace(",", ""))
                if exp != found:
                    errors.append({
                        "Table Name": table_name,
                        "Row Index": idx,
                        "Expected Value": exp_raw,
                        "Found Value": found_raw
                    })
            except:
                if match.group(2) != match.group(3):
                    errors.append({
                        "Table Name": table_name,
                        "Row Index": match.group(1),
                        "Expected Value": match.group(2),
                        "Found Value": match.group(3)
                    })

    return {"Table Name": table_name, "Status": "Correct" if not errors else "Wrong"}, errors


# --- Full Validation Pipeline ---
def run_validation(tables_dict: dict):
    summary, all_errors = [], []

    for name, df in tables_dict.items():
        status, errors = validate_table(name, df)
        summary.append(status)
        all_errors.extend(errors)

    summary_df = pd.DataFrame(summary)
    print("\n✅ Summary Report:")
    display(summary_df)

    if all_errors:
        error_df = pd.DataFrame(all_errors)
        print("\n❌ Detailed Error Report:")
        display(error_df)

# --- Example Usage ---
tables_dict = statement_table_extraction(pdf_path, df)
run_validation(tables_dict)


In [ ]:
import pandas as pd
import os
import re
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv
from IPython.display import display

# --- Load Environment Variables ---
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# --- Initialize LangChain GPT-4o Model ---
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=openai_api_key
)

# --- Output Containers ---
results = []
error_data = []

# --- Table Validation Function ---
def validate_table_with_langchain(table_name: str, df: pd.DataFrame):
    table_markdown = df.to_markdown(index=True, tablefmt="grid")

    prompt = f"""
You are a qualified financial auditor.

Your task is to **mathematically verify** the totals and subtotals in the following financial table: **{table_name}**.

### Your rules:
- ✅ ONLY check **Each row calculation logic** — no formatting issues like commas
- ✅ DO NOT mark a row as incorrect if the **Expected and Found values are exactly the same** (even if repeated).
- ❌ DO NOT include any row where values are equal.
- ✅ ONLY report rows where the **Expected ≠ Found**.
- ⚠️ If all calculations are correct, just say so.
---
### Output Format (strictly this format):

- If table is correct:
    Table Name: {table_name}, Status: correct

- If table has errors:
    Table Name: {table_name}, Status: wrong
    Table Name: {table_name}, Row Index: <row index>, Expected: <expected value>, Found: <found value>

📌 Notes:
- Use 0-based indexing (first row = index 0)
- Do not give any explanations.
- Do not show correct rows.
- Do not hallucinate mismatches — values must be **numerically different**.

---

### Table for Validation:
{table_markdown}

Begin validation:
""".strip()

    try:
        response = llm([HumanMessage(content=prompt)])
        result = response.content.strip()
        print(result)

        # Detect overall status
        status_match = re.search(r'Status\s*:\s*(correct|wrong)', result, re.IGNORECASE)
        status = status_match.group(1).capitalize() if status_match else "Unknown"

        # Extract row-level errors
        actual_errors = []

        if status.lower() == "wrong":
            row_matches = re.finditer(
                r'Row Index\s*:\s*(\d+),\s*Expected\s*:\s*([-,\d]+)[,|\s]+(?:Found|Expected)\s*:\s*([-,\d]+)',
                result, re.IGNORECASE
            )

            for match in row_matches:
                row_index = int(match.group(1))
                expected_str = match.group(2).replace(',', '')
                found_str = match.group(3).replace(',', '')

                try:
                    expected_val = float(expected_str)
                    found_val = float(found_str)

                    if expected_val == found_val:
                        continue  # ✅ Skip if values match

                    actual_errors.append({
                        "Table Name": table_name,
                        "Row Index": row_index,
                        "Expected Value": match.group(2),
                        "Found Value": match.group(3)
                    })
                except ValueError:
                    if expected_str != found_str:
                        actual_errors.append({
                            "Table Name": table_name,
                            "Row Index": row_index,
                            "Expected Value": match.group(2),
                            "Found Value": match.group(3)
                        })

        if status.lower() == "wrong" and not actual_errors:
            status = "Correct"

        results.append({"Table Name": table_name, "Status": status})

        if actual_errors:
            error_data.extend(actual_errors)

    except Exception as e:
        results.append({"Table Name": table_name, "Status": f"Validation Error: {str(e)}"})


# --- Run Validation for All Tables ---
tables_dict = statement_table_extraction(pdf_path, df)

for table_name, table_df in tables_dict.items():
    validate_table_with_langchain(table_name, table_df)

# --- Display Summary Report ---
summary_df = pd.DataFrame(results)
print("\n✅ Summary Validation Status:")
display(summary_df)

# --- Display Detailed Errors If Any ---
if error_data:
    error_df = pd.DataFrame(error_data)[["Table Name", "Row Index", "Expected Value", "Found Value"]]
    print("\n❌ Detailed Error Report:")
    display(error_df)


## Using GPT model with schema and outputparser

In [ ]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
 ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# --- Load .env credentials ---
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
base_url =os.getenv('https://api.groq.com/openai/v1')

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=api_key
)


# --- Output Schema ---
response_schemas = [
    ResponseSchema(name="status", description="correct or wrong"),
    ResponseSchema(name="errors", description="List of dictionaries with row index, expected and found values")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# --- Prompt Template ---
system_template = (
    "You are a qualified financial auditor. "
    "Check for only mathematical correctness of totals in a financial table. "
    "Respond in a structured format."
)

human_template = """
Table Name: {table_name}

Validation Rules:
- ✅ Only check calculations (ignore formatting like commas).
- ❌ Do not report correct rows.
- ✅ Report rows where Expected ≠ Found (numerically).
- ⚠️ If everything is correct, say so.
❗ Only include incorrect rows where numbers do not match after removing commas.

Format Instructions:
{format_instructions}

--- Table Data ---
{table_markdown}
"""

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

# --- Helper Function to evaluate expressions safely ---
def safe_eval(expr: str) -> float:
    # expr = expr.replace(",", "")
    return eval(expr, {"__builtins__": None}, {})

# --- Core Validator Function ---
def validate_table(table_name: str, df: pd.DataFrame):
    table_markdown = df.to_markdown(index=True, tablefmt="grid")
    messages = prompt_template.format_messages(
        table_name=table_name,
        table_markdown=table_markdown,
        format_instructions=format_instructions
    )

    response = llm.invoke(messages)
    result = response.content.strip()


    errors = []
    status = "Unknown"

    try:
        parsed = parser.parse(result)
        status = parsed["status"].capitalize()

        for err in parsed["errors"]:
            try:
                row_idx = int(err.get("Row Index", err.get("row_index", -1)))
                expected_raw = str(err.get("Expected", err.get("expected", "")))
                found_raw = str(err.get("Found", err.get("found", "")))

                # Remove commas and evaluate both
                expected = safe_eval(expected_raw)
                found = safe_eval(found_raw)

                if abs(expected - found) > 1e-2:
                    errors.append({
                        "Table Name": table_name,
                        "Row Index": row_idx,
                        "Expected Value": expected,
                        "Found Value": found
                    })
            except Exception as eval_err:
                print(f"⚠️ Eval failed for row {err}: {eval_err}")
                continue

    except Exception as e:
        print(f"⚠️ Structured parsing failed: {e}")
        status_match = re.search(r"Status\s*:\s*(correct|wrong)", result, re.IGNORECASE)
        status = status_match.group(1).capitalize() if status_match else "Unknown"

        if status.lower() == "wrong":
            row_pattern = r"(\d+)\s*[:,\-]?\s*([-+*/\d, ()]+)\s*[:=>\-]+?\s*([-+*/\d, ()]+)"
            for match in re.finditer(row_pattern, result, re.IGNORECASE):
                try:
                    idx = int(match.group(1))
                    expected_raw = match.group(2).strip()
                    found_raw = match.group(3).strip()

                    expected = safe_eval(expected_raw)
                    found = safe_eval(found_raw)

                    if abs(expected - found) > 1e-2:
                        errors.append({
                            "Table Name": table_name,
                            "Row Index": idx,
                            "Expected Value": expected,
                            "Found Value": found
                        })
                except Exception as eval_err:
                    print(f"⚠️ Eval fallback failed: {eval_err}")
                    continue

    return {"Table Name": table_name, "Status": "Correct" if not errors else "Wrong"}, errors

# --- Full Validation Pipeline ---
def run_validation(tables_dict: dict):
    summary, all_errors = [], []

    for table_name, df in tables_dict.items():
        status, errors = validate_table(table_name, df)
        summary.append(status)
        all_errors.extend(errors)

    summary_df = pd.DataFrame(summary)
    print("\n✅ Summary Report:")
    display(summary_df)

    if all_errors:
        error_df = pd.DataFrame(all_errors)[[
            "Table Name", "Row Index", "Expected Value", "Found Value"
        ]]
        print("\n❌ Detailed Error Report:")
        display(error_df)
    else:
        print("\n🎉 No errors found in any table!")

# --- Example Execution ---
# Ensure `statement_table_extraction(pdf_path, df)` is defined elsewhere
tables_dict = statement_table_extraction(pdf_path, df)
run_validation(tables_dict)
